# Website data extraction

In [ ]:
import pandas as pd

In [ ]:
dateDataFileNamePrefix = '2019-08-28_'
websiteDataFileNamePrefix = 'website_'
preQuestionnaireFileNameStem = 'AMR-Questionnaire-Pre'
postQuestionnaireFileNameStem = 'AMR-Questionnaire-Post'
gameQuestionnaireFileNameStem = 'Game-Questionnaire-Game'

# shared with 0.0 Data...
dataFolderPath = "../Data/"
csvSuffix = '.csv'

prequestionnaire = preQuestionnaireFileNameStem
postquestionnaire = postQuestionnaireFileNameStem
game1Questionnaire = gameQuestionnaireFileNameStem + '1'
game2Questionnaire = gameQuestionnaireFileNameStem + '2'
game3Questionnaire = gameQuestionnaireFileNameStem + '3'

def getFileName(stem):
    return dataFolderPath + dateDataFileNamePrefix + websiteDataFileNamePrefix + stem + csvSuffix

dataFilesStems = [prequestionnaire, postquestionnaire, game1Questionnaire, game2Questionnaire, game3Questionnaire]
dataFiles = [getFileName(stem) for stem in dataFilesStems]
dataFiles, dataFilesStems

In [ ]:
allNames = set()
completeParticipants = set()
for f in dataFiles:
    unanonymizedData = pd.read_csv(f,dtype=str)
    names = set(unanonymizedData['Name'])
    allNames = set.union(allNames, names)
    if len(completeParticipants) == 0:
        completeParticipants = names
    else:
        completeParticipants = completeParticipants and names

In [ ]:
allNames

In [ ]:
completeParticipants

In [ ]:
def getStreamlinedString(name):
    return str.lower(name).replace(" ","")

In [ ]:
allNames = pd.DataFrame(index=[])

for stem in dataFilesStems:
    
    fileName = getFileName(stem)
    unanonymizedData = pd.read_csv(fileName,dtype=str)
    names = set(unanonymizedData['Name'])
    
    for name in names:
        allNames.loc[getStreamlinedString(name), stem] = "done"
            
#    allNames = allNames.append(names, sort=True).transpose().drop(index='Name')
allNames = allNames.sort_index()
allNames = allNames.fillna("-")
allNames

In [ ]:
def getNameFromEmail(data, email):
    return data[data["Email"]==email]["Name"].values[0]
getNameFromEmail(unanonymizedData, "nidhiben.patel@cri-paris.org")

In [ ]:
email = "nidhiben.patel@cri-paris.org"
#getNameFromEmail(unanonymizedData, email)
unanonymizedData[unanonymizedData["Email"]==email]["Name"].values[0]

In [ ]:
allNames = pd.DataFrame(index=[])
allNames.index.name = "email"
debugOn = False

for stem in dataFilesStems:
    
    fileName = getFileName(stem)
    unanonymizedData = pd.read_csv(fileName,dtype=str)
    emails = set(unanonymizedData['Email'])
    
    for email in emails:
        allNames.loc[getStreamlinedString(email), "name"] = getNameFromEmail(unanonymizedData, email)
        allNames.loc[getStreamlinedString(email), stem] = "done"
            
#    allNames = allNames.append(names, sort=True).transpose().drop(index='Name')
allNames = allNames.sort_index()
allNames1 = allNames.copy()
#allNames = allNames.fillna("-")
#allNames

todrop = set()
for email in allNames.index:
    if email not in todrop:
        name = allNames.loc[email, "name"]
        otherEmails = set(allNames.index) - set([email])
        #if name in allNames.loc[otherEmails, "name"].values:
        #if getStreamlinedString(name) in [getStreamlinedString(n) for n in allNames.loc[otherEmails,"name"]]:
        #if getStreamlinedString(name) in \
        #[getStreamlinedString(n) for n in allNames.loc[otherEmails, "name"].values]:
        matches = allNames[allNames["name"].apply(getStreamlinedString)==getStreamlinedString(name)]
        if len(matches) > 1:
            if debugOn:
                print()
                print(name + ", " + email)
            for match in matches.index:
                if match != email:
                    if debugOn:
                        print("- addressing match " + match)
            # merge lines with same names
                    for stem in dataFilesStems:
                        matchValue = allNames.loc[match, stem]
                        if debugOn:
                            print("- - matchValue " + str(matchValue))
                        newValue = allNames.loc[email, stem]
                        if pd.isna(newValue):
                            newValue = allNames.loc[match, stem]
                        if debugOn:
                            print("- - newValue " + str(newValue))
                        oldValue = allNames.loc[email, stem]
                        if debugOn:
                            print("- - oldValue " + str(oldValue))
                        allNames.loc[email, stem] = newValue
                        if debugOn:
                            print("- - addressing column " + stem + " by putting value "\
                              + str(newValue) + " instead of " + str(oldValue))
                    if debugOn:
                        print("adding this to drop list: " + match)
                    todrop.add(match)
for elt in todrop:
    if debugOn:
        print("dropping " + elt)
    allNames = allNames.drop(elt)
allNames = allNames.fillna("")
allNames

In [ ]:
adr1 = "burunga.enock@yahoo.com"
adr2 = "rutaisire25@gmail.com"
adr3 = "rutaisuee@gmail.com"
allNames1.loc[adr1,:], allNames1.loc[adr2,:], allNames1.loc[adr3,:], allNames.loc[adr1,:]

In [ ]:
allNames1.loc[adr1, stem], allNames1.loc[adr2, stem], (allNames1.loc[adr2, stem] or allNames1.loc[adr1, stem])

In [ ]:
matches = allNames[allNames["name"].apply(getStreamlinedString)==getStreamlinedString("Burunga enock")]
for match in matches.index:
    print(match)

allNames[allNames["name"].apply(getStreamlinedString)==getStreamlinedString("Burunga enock")]

len(allNames[allNames["name"].apply(getStreamlinedString)==getStreamlinedString(name)])

In [ ]:
len(allNames)